In [1]:
import os

train_data_path = 'data/train/2025-04-11'
skip_files = ['stopwords']

def get_files(path: str) -> list:
    entries = os.listdir(path)
    return [os.path.join(path, entry) for entry in entries if os.path.isfile(os.path.join(path, entry))]

train_file_paths = get_files(train_data_path)
print(train_file_paths)

['data/train/2025-04-11\\1-相亲闪婚后，老公竟是千亿财阀.txt', 'data/train/2025-04-11\\10-腹黑娇妻：厉少，人设崩坏中.txt', 'data/train/2025-04-11\\100 小夫人一身反骨，大叔乖乖臣服.txt', 'data/train/2025-04-11\\101 墨爷的闪婚小甜妻.txt', 'data/train/2025-04-11\\102 流年不负你情深.txt', 'data/train/2025-04-11\\13-帝少甜婚令：娇妻，乖一点.txt', 'data/train/2025-04-11\\14-闪婚厚爱：病娇老公竟是千亿大佬.txt', 'data/train/2025-04-11\\17-闪婚三年不见面，再遇我是他秘书.txt', 'data/train/2025-04-11\\一夜惊喜：超级总裁喜当爹.txt', 'data/train/2025-04-11\\一胎六宝：总裁爸比超凶猛.txt', 'data/train/2025-04-11\\一胎双宝：季总你出局了.txt', 'data/train/2025-04-11\\一顾很倾情(www.sxcnw.org).txt', 'data/train/2025-04-11\\七年之痒，前夫不知道我重生了.txt', 'data/train/2025-04-11\\七爷的罪妻.txt', 'data/train/2025-04-11\\三宝联萌 大佬妈咪拽又飒.txt', 'data/train/2025-04-11\\三宝联萌：大佬妈咪拽又飒.txt', 'data/train/2025-04-11\\三年恋情曝光，我成了白月光的替身.txt', 'data/train/2025-04-11\\上善若玉(www.sxcnw.org).txt', 'data/train/2025-04-11\\不受宠？无所谓，杀疯豪门万人跪 (1~20章)-总裁、多重身份、傲慢霸气、复仇、男-有魅力的角色、浪漫、甜蜜、言情、大女主、甜宠.txt', 'data/train/2025-04-11\\不当舔狗后，肥妻她美又飒！ (1~20章)-多重身份、离婚、豪门霸总、真假千金、追妻火葬场、破镜重圆、甜宠、打脸.txt', 'd

In [2]:
from src.util.read_file import read_file

encoding = 'utf-8'
train_novels = list[tuple]()

for file_path in train_file_paths:
    try:
        train_novels.append((file_path, read_file(file_path, encoding)))
    except Exception:
        print('Decode Error ', file_path)

with open('data/stopwords', encoding='utf-8') as _f:
    stopwords = _f.readlines()
    stopwords = [x.replace('\n', '') for x in stopwords]
    
if len(train_novels) > 0:
    print(str(train_novels[0])[:512], '...')
print(stopwords[:128])

Decode Error  data/train/2025-04-11\一顾很倾情(www.sxcnw.org).txt
Decode Error  data/train/2025-04-11\七爷的罪妻.txt
Decode Error  data/train/2025-04-11\三宝联萌 大佬妈咪拽又飒.txt
Decode Error  data/train/2025-04-11\上善若玉(www.sxcnw.org).txt
Decode Error  data/train/2025-04-11\冷情总裁的退婚新娘.txt
Decode Error  data/train/2025-04-11\大佬你的小娇娇甜翻了.txt
Decode Error  data/train/2025-04-11\天才狂少的男妻.txt
Decode Error  data/train/2025-04-11\星河灿烂皆因你.txt
Decode Error  data/train/2025-04-11\暖婚急诊：商医生，来盖个章！.txt
('data/train/2025-04-11\\1-相亲闪婚后，老公竟是千亿财阀.txt', '\n第1章 来闪婚还摆架子\n书名： 相亲闪婚后，老公竟是千亿财阀 作者： 一板奶片 本章字数： 2164 更新时间： 2023-01-18 21:15:55\n\n“清溪，今天晚上你在外面吃个饭再回来了，没钱舅舅给你，你舅妈在发疯呢……”\n\n电话那头舅舅的话还没说完，又紧跟着传来舅妈尖锐的声音。\n\n“我发疯？你那贱种侄女今年26岁还在读书，不嫁人在我家吃我的用我的，还说我发疯？！现在公司遇到这么大的危机，我不管，你必须把她介绍给……”\n\n余清溪刚准备开口，电话匆忙地挂断了。\n\n她无声地叹了口气，看着自己刚发的奖学金，一份没动给舅舅转了过去。\n\n这样的情况，基本上每个星期都会出现。\n\n自从她妈妈去世后，她被舅舅接回家。因为舅妈不喜欢她，她只能在这个家小心翼翼地生活着，直到考上了文物修复学的研究生，她才将前些年欠舅舅的学费用奖学金还干净了。\n\n本想着，这个家无论如何她都得搬走了，苦于舅舅一直不同意，舅妈也因此一直跟他吵架。\n\n她唯一能想到能够正大光明搬走，舅舅不会担心的借口，就是结婚 ...
['!', '"', 

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from src.util.preprocess import split_words

with ThreadPoolExecutor(max_workers=20) as executor:
    train_seg_lists = list(executor.map(lambda _novel: (_novel[0], split_words(_novel[1], stopwords)), train_novels))
    
print(train_seg_lists[0])

Split: 4
Split: 4
Split: 4
Split: 4
Split: 5
Split: 3
Split: 4
Split: 3
Split: 4
Split: 3
Split: 4
Split: 4
Split: 3
Split: 4
Split: 8
Split: 8
Split: 4
Split: 5
Split: 4
Split: 12
These arent names: ['温氏家族成员', '魏氏家族成员', '男性', '女性']
Names Found: ['温诺', '温聆', '魏鹏']
These arent names: ['丈夫', '妻子', '老板', '总裁', '警察', '秘书', '助理', '爷爷', '奶奶', '爸爸', '妈妈', '哥哥', '弟弟', '姐姐', '妹妹', '宝宝']
Names Found: ['陆绾', '郝泽', '陆婷婷', '厉溟庭', '陆正风', '张峰', '鄢世祥', '萧辞']
These arent names: []
Names Found: ['温聆', '林泽', '明啸', '周炎', '祁烽', '秦苒', '温诺']
These arent names: ['人名']
Names Found: ['温聆', '祁烽', '林泽', '明啸', '周炎']
These arent names: ['无']
Names Found: ['江政和', '冉雨清']
These arent names: ['学生']
Names Found: ['叶星语', '封薄言', '谢青岑']
These arent names: ['学生']
Names Found: ['叶星语', '封薄言', '谢青岑']
These arent names: ['丈夫', '奶奶', '中年妇女', '警察', '总裁', '皇后', '宝宝']
Names Found: ['姜醒醒', '战夜枭', '阮安颜', '姜东林', '季长兰', '姜珍娇', '舒泽云']
These arent names: ['学生']
Names Found: ['唐知落', '梁远哲']
These arent names: []
Names Found: ['唐知落', '盛行舟',

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wuzihao\AppData\Local\Temp\jieba.cache


These arent names: ['无']
Names Found: ['温聆', '明啸', '周炎']
These arent names: ['女性', '人名']
Names Found: ['冉香玉', '冉雨清', '李桂花', '江政和']
These arent names: ['宝宝']
Names Found: ['程小莫', '马花花', '程依依', '程小暖', '季霆枫']
These arent names: []
Names Found: ['韩以沫', '江煜城', '沈曼音', '沈漫音', '徐西洲']
These arent names: []
Names Found: ['奕骁', '沈奕骁', '陆晓云', '黎月']
These arent names: []
Names Found: ['陆凛节', '蒋晓雪', '周容寒', '乔意', '肖筱', '肖明琛']
These arent names: []
Names Found: ['沈奕骁', '陆晓云', '孟子丞', '梁枫']
These arent names: ['女性', '男性', '名字']
Names Found: ['简诗', '孟美竹', '傅盛年', '简瑶', '顾湘']


Loading model cost 0.396 seconds.
Prefix dict has been built successfully.


These arent names: []
Names Found: ['周炎', '明啸', '温聆', '林泽', '祁烽', '温诺', '白芬', '温平']
Split: 4
Split: 4
These arent names: ['总裁']
Names Found: ['封墨霆']
These arent names: ['林氏家族成员', '学生', '同事', '朋友', '亲人', '同乡', '邻居']
Names Found: ['林筱乐', '胡天宇', '林小婉', '杜晴容', '方梅琴', '林正阳']
These arent names: ['学生']
Names Found: ['程依依', '王宣', '季霆枫', '季北', '江乐']
These arent names: ['无']
Names Found: ['叶星语', '苏颜颜', '封薄言', '谢青岑', '叶世华']
These arent names: ['女性', '人名']
Names Found: ['乔意', '陆凛节', '周容寒', '阿雪', '蒋晓雪', '徐秀华']
These arent names: ['妹妹']
Names Found: ['盛年', '简瑶', '乔妹']
These arent names: ['学生']
Names Found: ['叶星语', '封薄言', '苏颜颜', '许牧', '叶世华', '谢青岑']
Split: 4
These arent names: []
Names Found: ['姜醒醒', '王正文', '阮安颜', '战夜枭', '靳宴西', '秦司曜']
These arent names: ['学生']
Names Found: ['钟元萱', '陆勤勉']
These arent names: ['人名']
Names Found: ['姜醒醒', '战夜枭', '靳宴西', '王正文', '阮安颜']
These arent names: ['无']
Names Found: ['余清溪', '裴漠衍', '余晴雨', '张晴', '余雯雯']
These arent names:These arent names: ['学生']
Names Found: ['韩以沫', '徐西洲

In [ ]:
train_documents = [x[1] for x in train_seg_lists]

In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(train_documents)
corpus = [dictionary.doc2bow(x) for x in train_documents]

print(corpus[:16])

In [ ]:
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

num_topics = 5

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=64,
    random_state=32
)

cm = CoherenceModel(model=lda_model, texts=train_documents, coherence='u_mass')
coherence = cm.get_coherence()
print(f"Coherence={coherence}")
pyLDAvis.display(gensimvis.prepare(lda_model, corpus, dictionary))

In [ ]:
# save model
lda_model.save('model/lda/lda_model')